# Kapitola 11: Prostor stavu - Jak AI kresli mapu k reseni problemu

<div style="background: linear-gradient(90deg, #667eea 0%, #764ba2 100%); padding: 20px; border-radius: 10px; margin: 20px 0;">
    <h2 style="color: white; margin: 0;">Blok 2 | Algoritmy a hledani</h2>
    <p style="color: white; margin: 10px 0;">Stavovy prostor, BFS, vizualizace grafu</p>
</div>

## Co se naucite v teto kapitole

1. **Stavovy prostor** - Jak AI prevadi problemy na mapu
2. **Stavy a operatory** - Zakladni stavebni kameny
3. **BFS algoritmus** - Prohledavani do sirky
4. **Vizualizace grafu** - Jak nakreslit mapu problemu
5. **Prakticky projekt** - Problem s nadobami na vodu

---

In [ ]:
# Inicializace prostredi pro Google Colab
import sys
from datetime import datetime

print("=" * 60)
print("KAPITOLA 11: Prostor stavu")
print("=" * 60)
print(f"Python verze: {sys.version.split()[0]}")
print(f"Datum: {datetime.now().strftime('%Y-%m-%d %H:%M')}")
print("=" * 60)

# Instalace knihoven
!pip install -q networkx matplotlib

print("\nProstredi pripraveno!")

---

## Motivacni priklad: Problem s nadobami

Predstavte si tuto situaci:

> Mate dve nadoby - jednu na **4 litry** a druhou na **3 litry**.
> Potrebujete odmerit **presne 2 litry** vody.
> Jak to udelate?

Zkouseni nahodne by trvalo vecne. Potrebujeme **systematicky pristup** - mapu vsech moznych kroku.

In [ ]:
# Vizualizace problemu

print("PROBLEM S NADOBAMI")
print("=" * 50)
print()
print("   +-------+    +-----+")
print("   |       |    |     |")
print("   |  4L   |    | 3L  |")
print("   |       |    |     |")
print("   +-------+    +-----+")
print()
print("CIL: Odmerit presne 2 litry vody")
print()
print("Povolene akce:")
print("   1. Naplnit 4L nadobu")
print("   2. Naplnit 3L nadobu")
print("   3. Vylit 4L nadobu")
print("   4. Vylit 3L nadobu")
print("   5. Prelit z 4L do 3L")
print("   6. Prelit z 3L do 4L")

---

## Anatomie problemu: Stavy, Operatory, Cil

Abychom mohli problem prevest na mapu, musime definovat **tri klicove veci**:

### 1. Stav (State)
Snimek situace v danem okamziku. U nasich nadob je stav dvojice cisel:
- `(voda_v_4L, voda_v_3L)`
- Pocatecni stav: `(0, 0)` - obe prazdne
- Cilovy stav: `(2, x)` nebo `(x, 2)` - kdekoli 2 litry

### 2. Operatory (Actions)
Povolene akce, ktere nas presouvaji mezi stavy.

### 3. Stavovy prostor (State Space)
Kompletni mapa vsech stavu a prechodu mezi nimi = **GRAF**

In [ ]:
# Definice operatoru pro problem s nadobami

def get_next_states(state, cap_x=4, cap_y=3):
    """
    Vrati vsechny mozne nasledujici stavy z daneho stavu.
    
    Args:
        state: tuple (x, y) - aktualni stav
        cap_x: kapacita prvni nadoby (default 4)
        cap_y: kapacita druhe nadoby (default 3)
    
    Returns:
        list of (next_state, action_name)
    """
    x, y = state
    next_states = []
    
    # 1. Naplnit X (4L nadobu)
    if x < cap_x:
        next_states.append(((cap_x, y), "Naplnit 4L"))
    
    # 2. Naplnit Y (3L nadobu)
    if y < cap_y:
        next_states.append(((x, cap_y), "Naplnit 3L"))
    
    # 3. Vylit X
    if x > 0:
        next_states.append(((0, y), "Vylit 4L"))
    
    # 4. Vylit Y
    if y > 0:
        next_states.append(((x, 0), "Vylit 3L"))
    
    # 5. Prelit z X do Y
    if x > 0 and y < cap_y:
        pour = min(x, cap_y - y)  # Kolik muzeme prelit
        next_states.append(((x - pour, y + pour), "Prelit 4L->3L"))
    
    # 6. Prelit z Y do X
    if y > 0 and x < cap_x:
        pour = min(y, cap_x - x)  # Kolik muzeme prelit
        next_states.append(((x + pour, y - pour), "Prelit 3L->4L"))
    
    return next_states

# Demonstrace
print("OPERATORY - MOZNE AKCE")
print("=" * 50)

test_states = [(0, 0), (4, 0), (1, 3), (4, 3)]

for state in test_states:
    print(f"\nZe stavu {state}:")
    for next_state, action in get_next_states(state):
        print(f"   {action} -> {next_state}")

---

## BFS: Prohledavani do sirky

**BFS (Breadth-First Search)** je algoritmus, ktery prohledava graf po vrstvach:

1. Zacni v pocatecnim stavu
2. Prozkoumej vsechny sousedy (vzdalenost 1)
3. Pak vsechny jejich sousedy (vzdalenost 2)
4. Opakuj, dokud nenajdes cil

**Vyhoda:** Najde vzdy **nejkratsi cestu** (nejmensi pocet kroku).

In [ ]:
from collections import deque

def bfs_solve(start, is_goal, get_neighbors):
    """
    Obecny BFS algoritmus.
    
    Args:
        start: pocatecni stav
        is_goal: funkce, ktera vraci True pokud je stav cilovy
        get_neighbors: funkce, ktera vraci sousedy stavu
    
    Returns:
        (path, visited_count) nebo (None, visited_count)
    """
    # Fronta: (stav, cesta k nemu)
    queue = deque([(start, [start])])
    visited = {start}
    
    while queue:
        current, path = queue.popleft()
        
        # Nasli jsme cil?
        if is_goal(current):
            return path, len(visited)
        
        # Pridej sousedy do fronty
        for neighbor, action in get_neighbors(current):
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append((neighbor, path + [neighbor]))
    
    return None, len(visited)

# Test BFS na problemu s nadobami
print("BFS RESENI PROBLEMU S NADOBAMI")
print("=" * 50)

# Cilova funkce: 2 litry v kterekoli nadobe
def is_goal(state):
    return state[0] == 2 or state[1] == 2

# Reseni
path, visited_count = bfs_solve(
    start=(0, 0),
    is_goal=is_goal,
    get_neighbors=lambda s: get_next_states(s)
)

if path:
    print(f"\nReseni nalezeno v {len(path)-1} krocich!")
    print(f"Prozkoumano {visited_count} stavu.")
    print("\nCesta:")
    for i, state in enumerate(path):
        x, y = state
        print(f"   {i}. ({x}L, {y}L)")
else:
    print("Reseni neexistuje!")

In [ ]:
# Detailni vypis reseni s akcemi

def solve_with_actions(start=(0, 0), goal_amount=2):
    """Resi problem a vraci cestu s popisem akci."""
    
    queue = deque([(start, [(start, "Start")])])
    visited = {start}
    
    while queue:
        current, path = queue.popleft()
        
        if current[0] == goal_amount or current[1] == goal_amount:
            return path
        
        for next_state, action in get_next_states(current):
            if next_state not in visited:
                visited.add(next_state)
                queue.append((next_state, path + [(next_state, action)]))
    
    return None

# Reseni s akcemi
print("DETAILNI RESENI")
print("=" * 50)

solution = solve_with_actions()

if solution:
    print("\nKrok po kroku:")
    print()
    for i, (state, action) in enumerate(solution):
        x, y = state
        print(f"   Krok {i}: {action}")
        print(f"           4L: {'#' * x}{'.' * (4-x)} ({x}L)")
        print(f"           3L: {'#' * y}{'.' * (3-y)} ({y}L)")
        print()

---

## Vizualizace stavoveho prostoru

Nyni si nakreslime kompletni mapu (graf) problemu:

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

def build_state_space_graph(start=(0, 0), cap_x=4, cap_y=3, goal_amount=2):
    """
    Vytvori graf stavoveho prostoru a najde reseni.
    """
    G = nx.DiGraph()
    
    # BFS pro prozkoumani celeho prostoru
    queue = deque([start])
    visited = {start}
    solution_path = None
    paths = {start: [start]}
    
    while queue:
        current = queue.popleft()
        G.add_node(str(current))
        
        # Zkontroluj cil
        if solution_path is None and (current[0] == goal_amount or current[1] == goal_amount):
            solution_path = paths[current]
        
        for next_state, action in get_next_states(current, cap_x, cap_y):
            G.add_edge(str(current), str(next_state))
            
            if next_state not in visited:
                visited.add(next_state)
                paths[next_state] = paths[current] + [next_state]
                queue.append(next_state)
    
    return G, solution_path

# Vytvoreni grafu
G, solution_path = build_state_space_graph()

print("STATISTIKY STAVOVEHO PROSTORU")
print("=" * 50)
print(f"Pocet stavu (uzlu): {G.number_of_nodes()}")
print(f"Pocet prechodu (hran): {G.number_of_edges()}")
print(f"Delka reseni: {len(solution_path) - 1} kroku")

In [ ]:
# Vizualizace grafu

def visualize_state_space(G, solution_path, goal_amount=2):
    """
    Vizualizuje stavovy prostor s oznacenim startu, cilu a cesty.
    """
    plt.figure(figsize=(14, 10))
    
    # Rozlozeni grafu
    pos = nx.spring_layout(G, seed=42, k=2)
    
    # Barvy uzlu
    node_colors = []
    for node in G.nodes():
        state = eval(node)
        if state == (0, 0):
            node_colors.append('#2ecc71')  # Zelena - start
        elif state[0] == goal_amount or state[1] == goal_amount:
            node_colors.append('#e74c3c')  # Cervena - cil
        elif state in solution_path:
            node_colors.append('#f39c12')  # Oranzova - na ceste
        else:
            node_colors.append('#3498db')  # Modra - ostatni
    
    # Kresleni
    nx.draw(G, pos, 
            with_labels=True, 
            node_size=1500, 
            node_color=node_colors,
            font_size=8,
            font_weight='bold',
            arrows=True,
            arrowsize=10,
            edge_color='#bdc3c7',
            width=0.5)
    
    # Zvyrazneni cesty k reseni
    if solution_path:
        path_edges = [(str(solution_path[i]), str(solution_path[i+1])) 
                      for i in range(len(solution_path)-1)]
        nx.draw_networkx_edges(G, pos, 
                               edgelist=path_edges,
                               edge_color='#e74c3c',
                               width=3,
                               arrows=True,
                               arrowsize=15)
    
    plt.title("Stavovy prostor - Problem s nadobami\n" + 
              "Zelena=Start, Cervena=Cil, Oranzova=Cesta k reseni",
              fontsize=14)
    plt.tight_layout()
    plt.show()

# Vizualizace
visualize_state_space(G, solution_path)

---

## Interaktivni simulace

Zkuste si problem vyresit rucne:

In [ ]:
def interactive_jug_solver():
    """
    Interaktivni simulace problemu s nadobami.
    """
    x, y = 0, 0  # Pocatecni stav
    cap_x, cap_y = 4, 3
    goal = 2
    moves = 0
    
    print("INTERAKTIVNI SIMULATOR")
    print("=" * 50)
    print(f"Cil: Odmerit {goal} litry vody")
    print("Prikazy: 1-6 (viz akce), 'q' pro ukonceni")
    print()
    
    while True:
        # Zobraz stav
        print(f"\nAktualni stav: 4L={x}L, 3L={y}L  (tahy: {moves})")
        print(f"   4L: {'#' * x}{'.' * (cap_x-x)}")
        print(f"   3L: {'#' * y}{'.' * (cap_y-y)}")
        
        # Kontrola vitezstvi
        if x == goal or y == goal:
            print(f"\n*** VITEZSTVI! Dosahli jste {goal} litru v {moves} tazich! ***")
            break
        
        # Mozne akce
        print("\nMozne akce:")
        actions = get_next_states((x, y), cap_x, cap_y)
        for i, (state, action) in enumerate(actions, 1):
            print(f"   {i}. {action} -> {state}")
        
        # Vstup
        choice = input("\nVyberte akci (1-6) nebo 'q': ").strip()
        
        if choice.lower() == 'q':
            print("Ukonceno.")
            break
        
        try:
            idx = int(choice) - 1
            if 0 <= idx < len(actions):
                (x, y), action = actions[idx]
                moves += 1
                print(f"   -> {action}")
            else:
                print("   Neplatna volba!")
        except ValueError:
            print("   Zadejte cislo 1-6 nebo 'q'")

# Pro spusteni odkomentujte:
# interactive_jug_solver()

---

## Dalsi priklady: Jiny hlavolam

Zkusme BFS na jiny problem - **posuvne puzzle 3x3** (zjednodusena verze):

In [ ]:
# Jednoduchy priklad: 2x2 posuvne puzzle

def get_puzzle_neighbors(state):
    """
    Vrati mozne tahy pro 2x2 puzzle.
    State je tuple: (a, b, c, d) kde 0 = prazdne misto
    
    Layout:
    [0][1]
    [2][3]
    """
    state = list(state)
    empty_idx = state.index(0)
    neighbors = []
    
    # Mozne tahy podle pozice prazdneho mista
    moves = {
        0: [1, 2],      # Prazdne vlevo nahore: muze jit vpravo nebo dolu
        1: [0, 3],      # Prazdne vpravo nahore: muze jit vlevo nebo dolu
        2: [0, 3],      # Prazdne vlevo dole: muze jit nahoru nebo vpravo
        3: [1, 2]       # Prazdne vpravo dole: muze jit nahoru nebo vlevo
    }
    
    for swap_idx in moves[empty_idx]:
        new_state = state.copy()
        new_state[empty_idx], new_state[swap_idx] = new_state[swap_idx], new_state[empty_idx]
        neighbors.append((tuple(new_state), f"Posun {state[swap_idx]}"))
    
    return neighbors

def print_puzzle(state):
    """Vytiskne puzzle."""
    s = state
    print(f"   [{s[0] if s[0] else ' '}][{s[1] if s[1] else ' '}]")
    print(f"   [{s[2] if s[2] else ' '}][{s[3] if s[3] else ' '}]")

# Test
print("2x2 POSUVNE PUZZLE")
print("=" * 50)

# Pocatecni a cilovy stav
start_puzzle = (1, 3, 0, 2)  # Zamichane
goal_puzzle = (1, 2, 3, 0)    # Serazene

print("\nPocatecni stav:")
print_puzzle(start_puzzle)

print("\nCilovy stav:")
print_puzzle(goal_puzzle)

# Reseni pomoci BFS
path, visited = bfs_solve(
    start=start_puzzle,
    is_goal=lambda s: s == goal_puzzle,
    get_neighbors=get_puzzle_neighbors
)

if path:
    print(f"\nReseni nalezeno v {len(path)-1} krocich!")
    print("\nCesta:")
    for i, state in enumerate(path):
        print(f"\nKrok {i}:")
        print_puzzle(state)
else:
    print("\nReseni neexistuje!")

---

## Cviceni

Procvicte si prace se stavovym prostorem:

In [ ]:
# CVICENI 1: Jine kapacity nadob
# ================================
# Zkuste vyresit problem s jinymi kapacitami

print("CVICENI 1: Jine kapacity")
print("=" * 50)

# Upravte nasledujici parametry:
cap_x = 5  # Kapacita prvni nadoby
cap_y = 3  # Kapacita druhe nadoby
goal = 4   # Cilove mnozstvi

# Vlastni get_neighbors s jinymi kapacitami
def custom_neighbors(state):
    return get_next_states(state, cap_x, cap_y)

path, visited = bfs_solve(
    start=(0, 0),
    is_goal=lambda s: s[0] == goal or s[1] == goal,
    get_neighbors=custom_neighbors
)

print(f"\nKapacity: {cap_x}L a {cap_y}L, Cil: {goal}L")
if path:
    print(f"Reseni: {' -> '.join(str(s) for s in path)}")
    print(f"Pocet kroku: {len(path)-1}")
else:
    print("Reseni neexistuje!")

In [ ]:
# CVICENI 2: Pocitani stavu
# ==========================
# Spocitejte, kolik ruznych stavu existuje

print("CVICENI 2: Analyza stavoveho prostoru")
print("=" * 50)

def count_all_states(cap_x, cap_y):
    """Spocita vsechny dosazitelne stavy."""
    visited = set()
    queue = deque([(0, 0)])
    visited.add((0, 0))
    
    while queue:
        current = queue.popleft()
        for next_state, _ in get_next_states(current, cap_x, cap_y):
            if next_state not in visited:
                visited.add(next_state)
                queue.append(next_state)
    
    return visited

# Analyza pro ruzne kapacity
test_cases = [(4, 3), (5, 3), (5, 2), (7, 5)]

print("\nKapacity | Pocet stavu | Max. moznych")
print("-" * 45)

for cap_x, cap_y in test_cases:
    states = count_all_states(cap_x, cap_y)
    max_possible = (cap_x + 1) * (cap_y + 1)
    print(f"  {cap_x}L, {cap_y}L   |     {len(states):2d}      |     {max_possible:2d}")

In [ ]:
# CVICENI 3: Vlastni problem
# ===========================
# Definujte vlastni problem a vyresite ho pomoci BFS

print("CVICENI 3: Preprava vlka, kozy a zeli")
print("=" * 50)
print("""
Klasicky hlavolam:
- Farmář chce prevézt vlka, kozu a zeli pres reku
- Lod unese jen farmáre + 1 vec
- Vlk sezere kozu, kdyz je nechate same
- Koza sezere zeli, kdyz je nechate same

Stav: (farmář, vlk, koza, zelí) - kde je kazdy (0=levy breh, 1=pravy breh)
""")

def is_safe(state):
    """Kontroluje, zda je stav bezpecny."""
    farmer, wolf, goat, cabbage = state
    
    # Vlk a koza bez farmáre
    if wolf == goat and farmer != wolf:
        return False
    
    # Koza a zeli bez farmáre
    if goat == cabbage and farmer != goat:
        return False
    
    return True

def get_river_neighbors(state):
    """Vrati mozne tahy pro problem s prepravu."""
    farmer, wolf, goat, cabbage = state
    neighbors = []
    
    new_farmer = 1 - farmer  # Farmár prejizdi na druhou stranu
    
    # Farmár jede sam
    new_state = (new_farmer, wolf, goat, cabbage)
    if is_safe(new_state):
        neighbors.append((new_state, "Farmár jede sám"))
    
    # Farmár veze vlka
    if wolf == farmer:
        new_state = (new_farmer, new_farmer, goat, cabbage)
        if is_safe(new_state):
            neighbors.append((new_state, "Farmár veze vlka"))
    
    # Farmár veze kozu
    if goat == farmer:
        new_state = (new_farmer, wolf, new_farmer, cabbage)
        if is_safe(new_state):
            neighbors.append((new_state, "Farmár veze kozu"))
    
    # Farmár veze zelí
    if cabbage == farmer:
        new_state = (new_farmer, wolf, goat, new_farmer)
        if is_safe(new_state):
            neighbors.append((new_state, "Farmár veze zelí"))
    
    return neighbors

# Reseni
start_river = (0, 0, 0, 0)  # Vse na levem brehu
goal_river = (1, 1, 1, 1)   # Vse na pravem brehu

path, visited = bfs_solve(
    start=start_river,
    is_goal=lambda s: s == goal_river,
    get_neighbors=get_river_neighbors
)

if path:
    print(f"\nReseni nalezeno v {len(path)-1} krocich!")
    print("\nKrok po kroku (F=Farmár, V=Vlk, K=Koza, Z=Zelí):")
    print("L=Levy breh, P=Pravy breh")
    print()
    
    for i, state in enumerate(path):
        f, w, g, c = state
        left = []
        right = []
        
        if f == 0: left.append('F')
        else: right.append('F')
        if w == 0: left.append('V')
        else: right.append('V')
        if g == 0: left.append('K')
        else: right.append('K')
        if c == 0: left.append('Z')
        else: right.append('Z')
        
        print(f"   {i}. L:[{','.join(left) or '-'}] ~~~~~ P:[{','.join(right) or '-'}]")

---

## Shrnuti kapitoly

<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 30px; border-radius: 10px; color: white; margin: 30px 0;">
    <h3 style="color: white; margin-top: 0;">Co jste se naucili</h3>
    <ul style="list-style: none; padding-left: 0;">
        <li>* Stavovy prostor = mapa vsech moznych stavu problemu</li>
        <li>* Stav = snimek situace (napr. (4L, 2L))</li>
        <li>* Operator = akce, ktera meni stav</li>
        <li>* BFS = prohledavani do sirky, najde nejkratsi cestu</li>
        <li>* Graf = vizualni reprezentace stavoveho prostoru</li>
    </ul>
    
    <h3 style="color: white;">Klicove dovednosti</h3>
    <p>Dokazete prevest problem na graf a najit reseni pomoci BFS.</p>
    
    <h3 style="color: white;">Dalsi kapitola</h3>
    <p>Kapitola 12: DFS a porovnani prohledavacich strategii</p>
</div>

---

*Notebook pro kurz AI - Blok 2: Algoritmy a hledani*  
*Kompatibilni s Google Colab*